# Analyzing archived NEXRAD data
`Replace the completely random, authentic, and not-fake names below with your group member names`
- Onesie McOne
- Twosie McTwo
- Threesie McThree
- Foursie McFour

## Introdution - We'll do this two ways
The National Centers for Environmental Information (NCEI) archives radar data onto Amazon S3 servers. Anyone with a computer and the internet can download the data. That's a lot of data at your fingertips!

Reading data in the cloud is a great way to do research without filling up your computer's storage. Here we'll do it two ways.

1. First, we'll use Amazon's software to access the data, which we will read using MetPy.
2. Second, we'll use Siphon and NCEI's THREDDS data server to access the data.

I don't really understand how either of these works, honestly, and it's a shame. There's lots of great data out there that I could use more easily if I had a good feel for THREDDS servers in particular.

What I can tell you is THREDDS is really powerful and handy. I'll explain using an example. Suppose you have a really big data file online, representing some global variable every six hours for a whole month. You are only interested in the most recent data over Indiana. Rather than download the entire file, you can ask the THREDDS server to read the file's metadata and figure out which part of the file you need. The THREDDS server will then extract that piece of the file, and package it (with the original file's metadata) in a new file for you to download. That saves a lot of time, storage, and internet bandwidth!

In [ ]:
from boto3 import resource as boto3_resource
from botocore import UNSIGNED as botocore_UNSIGNED
from botocore.client import Config
from siphon.radarserver import RadarServer
import matplotlib.pyplot as plt
from metpy.io import Level2File
import metpy.plots as mpplots # We import MetPy and use it to get the colortable and value mapping information for the NWS Reflectivity data.
from datetime import datetime, timedelta

'''We define some helper functions to make working with the data easier.
The first takes the raw data and converts it to floating point values with
the missing data points appropriately marked.
The second helps with converting the polar coordinates (azimuth and range)
to Cartesian (x and y).
The third helps with making a map to keep things simpler later. 
The fourth and fifth help me copy the colormaps in Figure 2 of
Zhang et al., (2019)
'''
from radar_lab_helper_functions import *
%matplotlib inline

# Part 1. MetPy, Amazon Web Services, and the Maysville tornado
You will want to review the following:
- [training course section on Base Reflectivity](https://training.weather.gov/wdtd/courses/rac/products/z/story_html5.html)
- [training course section on Base Velocity](https://training.weather.gov/wdtd/courses/rac/products/v-srm/story_html5.html)
- [training course section on Specific Differential Phase](https://training.weather.gov/wdtd/courses/rac/products/kdp/story_html5.html)
- [training course section on Differential Reflectivity](https://training.weather.gov/wdtd/courses/rac/products/zdr/story_html5.html)
- [training course section on Base Spectrum Width](https://training.weather.gov/wdtd/courses/rac/products/sw/presentation_html5.html)
- [training course section on Correlation Coefficient](https://training.weather.gov/wdtd/courses/rac/products/cc/story_html5.html)

In [ ]:
print('Starting the Amazon Web Services Software Development Kit...')
s3 = boto3_resource('s3', config=Config(signature_version=botocore_UNSIGNED,
                                        user_agent_extra='Resource'))
print("Accessing NOAA's data (see https://noaa-nexrad-level2.s3.amazonaws.com/index.html)...")
bucket = s3.Bucket('noaa-nexrad-level2') # 
for obj in bucket.objects.filter(Prefix='2021/12/11/KPAH/KPAH20211211_033529_V06'):
    print(obj)

    print("Reading the Level 2 data using MetPy...")
    f = Level2File(obj.get()['Body'])

# I'm hiding a bunch of stuff in here so the notebook isn't gigantic,
# and because I don't want to give way the answer to the question below.
plot_six_panel(f, title='KPAH Level 2 Data, local time 21:35:29 on 10 December 2021')

## Problem
1. What's with all the white space to the northwest and southeast of the radar?
2. What is the prevailing wind direction? How can you tell?
3. Record below the approximate coordinates of the tornado (with the radar being at 0,0) of the Maysville tornado t the time of this scan.
4. Describe how you can tell there is a tornado here, referring to at least three of the panels above.
5. In rectangular coordinates, identify two regions:
    - a region of large dropsize, and
    - a region with heavy precipitation but smaller drop size.
6. What is the mystery variable in panel (d)? How can you tell?
7. In rectangular coordinates, identify what you believe to be neighboring neighboring areas of warm rain and cold rain.
8. Ask a weather student in the group to speculate on why warm rain and cold rain might occur near to each other in a tornadic storm. Record their answer.
9. What do you think causes the spike in spectrum width?

`enter your responses here`

## Clean up!
Some of the variables in this notebook are big enough to cause trouble with our free server. Run the cell below to delete the variables from the above exercise.

In [ ]:
del(s3, bucket, obj, f)

# Part 2. Siphon, THREDDS, and Hurricane Maria
This notebook shows how to access the THREDDS Data Server (TDS) instance that is serving up archived NEXRAD Level 2 data hosted on Amazon S3. The TDS provides a mechanism to query for available data files, as well as provides access to the data as native volume files, through OPeNDAP, and using its own CDMRemote protocol. Since we're using Python, we can take advantage of Unidata's Siphon package, which provides an easy API for talking to THREDDS servers.

## But first, downloading the single latest volume. Just FYI

In [ ]:
# First we'll create an instance of RadarServer to point to the appropriate radar server access URL.
rs = RadarServer('http://tds-nexrad.scigw.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
query = rs.query()
#Next, we'll create a new query object to help request the data.
# Using the chaining methods, let's ask for the latest data at the radar KIND.
# We see that when the query is represented as a string, it shows the encoded URL.
print('The latest dataset available is {}'.format(query.stations('KIND').time(datetime.utcnow())))

# We can use the RadarServer instance to check our query, to make sure we have required parameters and that we have chosen valid station(s) and variable(s)
print('The dataset looks good: {}'.format(rs.validate_query(query)))

# Make the request, which returns an instance of TDSCatalog;
# This handles parsing the returned XML information.
catalog = rs.get_catalog(query)

# We can look at the datasets on the catalog to see what data we found by the query.
# We find one volume in the return, since we asked for the volume nearest to a single time.
print('Files matching our query: {}'.format(catalog.datasets))

# We can grab that dataset and call `remote_access()`, which sets us up to access the data remotely,
# WITHOUT DOWNLOADING THE ENTIRE FILE!
print('Getting access to the remote file...')
data = catalog.datasets[0].remote_access()

# Here's the sweep thing again.
# This dataset only has 4 sweeps, with "elev" of about 0.5, 0.5, 1.5, and 1.5 respectively.
# What gives? Oh well, at least we can make a pretty picture.
sweep = 0

# The CDMRemote reader provides an interface that is almost identical to the usual python NetCDF interface.
# We pull out the variables we need for azimuth and range, as well as the data itself.
print('Downloading just what we want...')
ref_var = data.variables['Reflectivity_HI']
ref_data = ref_var[sweep]
rng = data.variables['distanceR_HI'][:]
az = data.variables['azimuthR_HI'][sweep]

# Then convert the raw data to floating point values and the polar coordinates to Cartesian.
print('Converting to cartesian coordinates...')
ref = raw_to_masked_float(ref_var, ref_data)
x, y = polar_to_cartesian(az, rng)

# Use the function to make a new map and plot a colormapped view of the data
print('Plotting...')
ref_norm, ref_cmap = mpplots.ctables.registry.get_with_steps('NWSReflectivity', 5, 5)
fig = plt.figure(figsize=(10, 10))
ax = new_map(fig, data.StationLongitude, data.StationLatitude)
ax.pcolormesh(x, y, ref, cmap=ref_cmap, norm=ref_norm, zorder=0)

## Clean up!
Run the cell below to free up memory after making that plot.

In [ ]:
del(rs, query, catalog, data, sweep, ref_var, ref_data, rng, az, ref, x, y, ref_norm, ref_cmap, fig, ax)

## Finally, on to Hurricane Maria

This time we'll make a query based on a longitude, latitude point and using a time range.

In [ ]:
#Grab the first dataset so that we can get the longitude and latitude
rs = RadarServer('http://tds-nexrad.scigw.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
query = rs.query()
dt = datetime(2017, 9, 20, 10) # Our specified time
query.lonlat_point(-66., 18.3).time_range(dt - timedelta(hours=1.15), dt)
print("\nThe specified longitude, "\
      "latitude are in Puerto Rico and the THREDDS helpfully finds the closest station to that point. "\
      "The time range we request is an hour of data form 20 September 2017; "\
      "we're looking for data from Hurricane Maria. "\
      "We can see that this time we obtained multiple datasets.")
cat = rs.get_catalog(query)
print('Collecting reflectivity meshes...')
fig, meshes = animate_radar_loop(cat, vel=False)
print('Creating animation...')
ArtistAnimation(fig, meshes)

In [ ]:
print('Collecting velocity meshes...')
fig, meshes = animate_radar_loop(cat, vel=True)
print('Creating animation...')
ArtistAnimation(fig, meshes)

## Clean up!
Those animations use a bunch of memory. Let's clean up what we can.

In [ ]:
del(rs, query, dt, cat, fig, meshes)

## A closer look

In [ ]:
s3 = boto3_resource('s3', config=Config(signature_version=botocore_UNSIGNED,
                                        user_agent_extra='Resource'))
bucket = s3.Bucket('noaa-nexrad-level2') # 
for obj in bucket.objects.filter(Prefix='2017/09/20/TJUA/TJUA20170920_094526_V06'):
    f = Level2File(obj.get()['Body'])
plot_six_panel(f, title='TJUA Level 2 Data, local time 05:45:26 on 20 September 2017')

# Problem
1. Calculate the maximum wind speed in panel (e) above in mph. Describe how you came to your answer.
2. Are the rain drops in Hurricane Maria big or smaller than in the Kentucky storm above? How can you tell? Why would a marine storm have bigger droplets than a continental storm?

`put your answer in here`

## Clean up

In [ ]:
del(s3, bucket, obj, f)

## Problem
Using either the THREDDS method or the Siphon method, find a level 2 data file for a location and time of your choice, and run the six-panel analysis and plotter on that file. Interpret the findings in the space provided below.

To find the data, you can use either two approaches.
1. If you have a year, month, day, and station (which you can look up [here](http://climateviewer.org/history-and-science/atmospheric-sensors-and-emf-sites/maps/nexrad-doppler-radar-stations/)), you can get a list of files by using the amazon bucket filter thing to get a list of files:

`for obj in bucket.objects.filter(Prefix='2021/12/11/KIND/'):
    print(obj)`
    
2. If you have a longitude, latitude, and time, you can use Siphon instead, but I get weird answers. I feel like I was lucky to get Puerto Rico's coordinates this way. This example should give me Indianapolis, but it gives me Key West, instead:

`dt = datetime(2021, 12, 11, 12)
query.lonlat_point(-86.3, 39.7).time_range(dt - timedelta(hours=12), dt + timedelta(hours=12))
print(cat.datasets)`

Put your interpretation in the space provided below your code.

In [ ]:
s3 = boto3_resource('s3', config=Config(signature_version=botocore_UNSIGNED,
                                        user_agent_extra='Resource'))
bucket = s3.Bucket('noaa-nexrad-level2') # 

# example prefixes from above
# filename = '2021/12/11/KPAH/KPAH20211211_033529_V06
# filename = '2017/09/20/TJUA/TJUA20170920_094526_V06'
# a dummy prefix for you to change
filename = 'YYYY/MM/DD/NAME/NAMEYYYYMMDD_HHMMSS_V06'
for obj in bucket.objects.filter(Prefix=filename):
    f = Level2File(obj.get()['Body'])
plot_six_panel(f, title='ENTER YOUR OWN SNAZZY TITLE HERE')

`Enter your answer to the above problem here`

# Great work!

This lab just barely touches on...well, a lot of things, whether it's remote data access, or radar. Here are some of the links I used or check out. If you have time in your life, go through some.

This lab is adapted from several others online, including
- http://unidata.github.io/python-gallery/examples/Nexrad_S3_Demo.html
- https://gist.github.com/dopplershift/356f2e14832e9b676207

For more on reading NOAA data from amazon web services, see:
- https://docs.opendata.aws/noaa-nexrad/readme.html

On finding files on amazon web services using boto's bucket filter stuff, see:
- https://techoverflow.net/2021/03/08/how-to-filter-for-objects-in-a-given-s3-directory-using-boto3/

Some other references
- https://nbviewer.org/github/ARM-DOE/notebooks/blob/master/ASR_PI_2014/ARM%20NetCDF%20and%20Python%20Tutorial.ipynb (netCDF-based)
- https://github.com/openradar/AMS-Short-Course-on-Open-Source-Radar-Software/blob/master/9a_CSU_RadarTools_Demo-AMS_OSRSC.ipynb (also netCDF-based)

May be working this in more
- https://nbviewer.org/gist/dopplershift/356f2e14832e9b676207

More on Siphon here
+ [latest Siphon documentation](http://siphon.readthedocs.org/en/latest/)
+ [Siphon github repo](https://github.com/Unidata/siphon)
+ [TDS documentation](http://www.unidata.ucar.edu/software/thredds/current/tds/TDS.html)
+ metpy on [readthedocs](http://metpy.readthedocs.org) and [github](http://github.com/MetPy/MetPy)
+ [interactive radar station lookup](http://climateviewer.org/history-and-science/atmospheric-sensors-and-emf-sites/maps/nexrad-doppler-radar-stations/)

Colorbar instructions and code adapted from [Guangyuan(Frank) Li](https://github.com/frankligy) for the tutorial on the colorbar. Also used Adobe's [color gradient web app](https://color.adobe.com/create/image-gradient).

I stole colorbars from this recent paper, which may be a good radar read:
- Zhang, G., Mahale, V.N., Putnam, B.J. et al. Current Status and Future Challenges of Weather Radar Polarimetry: Bridging the Gap between Radar Meteorology/Hydrology/Engineering and Numerical Weather Prediction. Adv. Atmos. Sci. 36, 571–588 (2019). https://doi.org/10.1007/s00376-019-8172-4

There's some good stuff in these slides
- [Louisville, KY on dual-polarization radar](https://www.weather.gov/media/lmk/soo/Dual_Pol_Overview.pdf)
